In [3]:
import syft as sy
import torch as th
import numpy as np

root_domain = sy.register_root(email="owner@myorg.com", password="owerpwd", domain_name="Trask's Node")



# create a compliance officer user
compliance = root_domain.register_compliance(
                                email="compliance@myorg.com", 
                                password="pwd123")



[2021-06-15T22:17:48.939776+0100][CRITICAL][logger]][62411] UnknownPrivateException has been triggered.


In [2]:
import transformers

# Load model
model = transformers.AutoModel.from_pretrained("distilbert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")
config = model.config

# Turn off dropout
model = model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model_ptr = model.tag('distilbert').send(root_domain)

In [10]:
tokenizer_ptr = tokenizer.tag('tokenizer').send(root_domain)

In [11]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 4f10644e5c26483ea934f690189878cf>,[],,<class 'torch.Tensor'>
1,<UID: eeeff0eb43d44a0480718568fda7c379>,"[#xor, #inputs, #data_01.csv]",,<class 'torch.Tensor'>
2,<UID: 309ae60256ce4ac3b461c3b948e3999f>,"[#xor, #targets, #data_01.csv]",,<class 'torch.Tensor'>
3,<UID: f9bf1df9ab484abaaf7e502d05deff28>,[distilbert],,<class 'transformers.models.distilbert.modelin...
4,<UID: d4c407442bd440aeba920df2c878b317>,[tokenizer],,<class 'transformers.tokenization_utils_fast.P...
5,<UID: 9ee2d8a8e8564c64bee4392d1ea170ca>,[tokenizer],,<class 'transformers.tokenization_utils_fast.P...
